<a href="https://colab.research.google.com/github/micah-shull/AI_Agents/blob/main/050_Inventory_Management_Agent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# 🧾 Inventory Management Agent

### 📦 Rethinking Software with Agents

This lecture explores how to move away from traditional procedural programming and instead use LLM-powered agents to manage complexity through **simple tools + intelligent reasoning**. The example here is an **Inventory Management System**.

---

## 🧠 Key Idea

Rather than writing complex inventory code yourself, you define:

* ✅ **Simple tools** that handle data operations (CRUD: Create, Read, Update, Delete)
* 🎯 **Clear goals** that express what the agent should achieve
* 🧠 The LLM-powered agent orchestrates how to use tools to meet the goal

---

## 🧰 Tools (Functions)

The system defines three core tools with decorators:

```python
@register_tool(description="Save an item to inventory")
def save_item(...): ...
```

These tools are **very simple**—they just access a dictionary (`inventory_db`) and do exactly one thing.

---

## 🎯 Goals

The agent is given a single, high-level goal:

```python
Goal(
    name="inventory_management",
    description="Maintain an accurate inventory of items with descriptions, condition, value, and tracking"
)
```

---

## 🧠 System Prompt Capability

You can further shape the agent’s behavior by adding a **system prompt** that acts like an instruction manual:

```python
SystemPromptCapability("""You are an expert inventory manager...""")
```

This allows you to encode nuanced behaviors like:

* Be conservative with value estimates
* Use consistent formatting
* Consider visual cues in descriptions

---

## 🗣️ Natural Interaction

The user speaks naturally:

> *"I have a pair of Air Jordans, red with some wear and a Jumpman logo."*

The agent figures out what tool to use and how to call it. Example:

```json
Action: save_item
{
  "item_name": "Air Jordan Basketball Shoes",
  "description": "Red colorway with iconic Jumpman logo",
  "condition": "Used - visible wear and slight discoloration",
  "estimated_value": 85.00
}
```

---

## 🔄 Extending the Agent

You can add features without rewriting your agent logic:

* New tools (like image analysis)
* New system instructions
* Expanded goals (e.g., search, remove items)

---

## 🌍 General Pattern

This agent architecture can be applied to many real-world cases:

* ✅ Document understanding
* ✅ Policy compliance
* ✅ Customer service
* ✅ Automation systems

All by sticking to the pattern:

> **Simple tools** + **Well-defined goals** + **LLM intelligence** = Scalable and adaptive software





## 🧠 Rethinking Software Architecture

Instead of building complex inventory logic, we can break down the system into **simple tools** and let the agent handle the complexity:

### 🛠️ Simple Tools That Focus on Data Operations

```python
@register_tool(description="Save an item to inventory")
def save_item(action_context: ActionContext,
              item_name: str,
              description: str,
              condition: str,
              estimated_value: float) -> dict:
    """Save a single item to the inventory database."""
    inventory = action_context.get("inventory_db")
    item_id = str(uuid.uuid4())
    
    item = {
        "id": item_id,
        "name": item_name,
        "description": description,
        "condition": condition,
        "estimated_value": estimated_value,
        "added_date": datetime.now().isoformat()
    }
    
    inventory[item_id] = item
    return {"item_id": item_id}
```

```python
@register_tool(description="Get all inventory items")
def get_inventory(action_context: ActionContext) -> List[dict]:
    """Retrieve all items in the inventory."""
    inventory = action_context.get("inventory_db")
    return list(inventory.values())
```

```python
@register_tool(description="Get specific inventory item")
def get_item(action_context: ActionContext, item_id: str) -> dict:
    """Retrieve a specific inventory item."""
    inventory = action_context.get("inventory_db")
    return inventory.get(item_id)
```

---

### 🧩 Why It Works

Notice how **simple** these tools are—they just handle basic **CRUD operations**.
The **intelligence** and orchestration come from the **agent’s goals** and **system prompt**, not from the functions themselves.

This separation of concerns leads to:

* Easier debugging and testing
* Stronger modularity and reuse
* Dynamic, flexible behavior powered by the LLM

---




### 🧠 **The LLM’s Role Is Strategy, Not Execution**

* The tools (like `save_item`, `get_inventory`) are *pure utility functions*.
* They **don't know the overall goal** — they just do a tiny job (like "save this thing" or "return this list").
* The **LLM**, guided by the **agent's loop**, chooses when and how to use them based on the **GOAL** and **memory of past steps**.

---

### 🚫 No, the LLM Doesn’t Modify the Tools

* The LLM **does not rewrite or modify tool code**.
* It only decides **which tool to call** and **what arguments to pass in**.
* All logic lives in the agent loop — not in the tool code.

---

### 🧩 Analogy: Agent as a Project Manager

* Tools are like individual workers: they each know how to do one task (e.g., file clerk, database reader).
* The LLM is the project manager: it reads the goal ("Organize inventory"), thinks, and says:

  > “First, get the current inventory... then find the most valuable items... now write a report... finally, save it.”

---

### ✅ Benefits of Keeping Tools Simple

| Simplicity Wins When...       | Because...                                        |
| ----------------------------- | ------------------------------------------------- |
| You want fast, testable code  | Tools are independent and easy to unit test       |
| You want flexible agents      | The LLM can mix/match tool calls in creative ways |
| You want to swap environments | Tools can run locally, in cloud, or across APIs   |






### 🧠 What Happens When the LLM “Wants” More Tools?

If the LLM realizes — through the agent loop — that it **cannot accomplish the goal with the available tools**, here’s what typically happens:

---

### ✅ 1. **It Tries to Work Around It**

The LLM will often attempt creative combinations or reuses of tools:

> “I don’t have a `filter_items_by_value` tool, so I’ll call `get_inventory`, then sort and filter manually in memory.”

This is possible **if the memory contains enough info**, or if the environment supports in-memory processing.

---

### ⚠️ 2. **It May Loop or Fail Gracefully**

If the LLM can’t figure out a workaround, it might:

* Keep calling the same tool with varied inputs hoping for progress
* Eventually **call a `terminate` tool** with a message like:

  > “I don’t have the ability to delete items. Please provide a `delete_item` tool.”

This feedback is **extremely valuable**: it tells *you*, the developer, what the LLM *wishes* it had.

---

### 🛠️ 3. **You Add New Tools — Then Re-run**

This is where the human/agent collaboration shines. You can:

* Add a new tool (e.g., `delete_item`)
* Decorate it
* Re-run the agent with the same goal

Now, the LLM can take a better path to success.

---

### 🔁 So No, the LLM Cannot Modify or Create Tools Itself (yet)

Agents do **not currently write Python functions** or alter their own environment at runtime. That’s still **your job**.

However, **multi-agent systems** or **self-reflective agents** *can* do this with assistance. For example:

* A *code-writing agent* generates a Python function
* A *supervisor agent* approves it
* You (the human) choose to accept and re-register it

---

### 🧩 Summary

| Situation                             | Agent Behavior                              |
| ------------------------------------- | ------------------------------------------- |
| Has enough tools                      | Executes goal step-by-step                  |
| Tools missing but goal still possible | Tries creative workarounds                  |
| Tools insufficient                    | Terminates with explanation or fails softly |
| Human adds tools                      | Agent gets smarter instantly on next run    |





## 🔄 Building Agents Is an Iterative, Agent-Assisted Process

### 1. **You Start with a Goal and a Small Set of Tools**

You define the goal(s) and register a few basic tools:

```python
goals = [Goal(...)]
@register_tool
def get_inventory(...): ...
@register_tool
def save_item(...): ...
```

Then you run:

```python
agent.run("Track my belongings in the garage.")
```

---

### 2. **The Agent Tries Its Best**

Inside the loop, the LLM:

* Reads the goal
* Looks at available tools
* Tries to sequence tool calls to make progress

If it **succeeds**, great! You just built an MVP agent with minimal effort.

If it **struggles**, the LLM will usually tell you:

> "I don’t have a way to delete an item."
> "I cannot group items by category."

That’s feedback — **like a debugging trace**, but written in natural language.

---

### 3. **You Improve the Agent Based on Feedback**

You now:

* Add a `delete_item` tool
* Add filtering, updating, or exporting tools
* Possibly restructure goals

You don't rework prompts — you **add capability**, not tweak instructions.

---

### 4. **Repeat Until It Feels Autonomous**

Each run becomes a test of the agent’s capabilities:

* Can it handle more scenarios?
* Is it confused by output format?
* Does it repeat itself or forget things?

You iterate on **tools, goals, and memory** — *not prompts*.

---

## 🧠 Yes — This *Is* Agent Debugging

But instead of:

* Stepping through breakpoints,
* Reading tracebacks, or
* Tweaking low-level conditionals,

You’re doing:

* Iterative tool design
* High-level behavioral refinement
* Goal-tuning via feedback




🎯 Agent development feels more like **collaborating with an intern** than wrestling with code. Here's a quick checklist to guide your iterative agent-debugging workflow:

---

## ✅ Agent Debugging & Iteration Checklist

### 👟 1. Start Simple

* [ ] Define 1–2 clear **Goals**
* [ ] Register a **minimal set of tools** (e.g., list, get, save)
* [ ] Keep your **environment and memory** basic to start

---

### 🧠 2. Run the Agent

* [ ] Observe the **LLM's thought process** ("I will list files...")
* [ ] See what tool calls it makes (from `agent.run()`)
* [ ] Pay attention to any hallucinated or missing tools

---

### 🛠️ 3. Capture Agent Feedback

Ask yourself:

* [ ] Did it attempt something it **wasn’t equipped to do**?
* [ ] Did it **fail** a goal it should have reached?
* [ ] Did it make **inefficient or repetitive** decisions?

---

### ➕ 4. Add or Refine Tools

* [ ] Register missing tools it hinted at
* [ ] Adjust parameter requirements (e.g., make a param optional)
* [ ] Add filtering, sorting, or summarizing actions if needed

---

### 🎯 5. Re-evaluate Goals (if needed)

* [ ] Make the goal more specific or broken into sub-goals
* [ ] Ensure the goal reflects the *user's intent* clearly

---

### ♻️ 6. Repeat the Loop

* [ ] Run the agent again
* [ ] Check if behavior improves
* [ ] Keep refining until you say: **“Yep, that’s autonomous!”**






## 🧠 Define the Agent's Goals

```python
goals = [
    Goal(
        name="inventory_management",
        description="""Maintain an accurate inventory of items including:
        - Detailed descriptions
        - Condition assessment
        - Value estimates
        - Historical tracking"""
    )
]
```

### 🔧 Create the Agent with Clear Instructions

```python
agent = Agent(
    goals=goals,
    agent_language=JSONAgentLanguage(),
    action_registry=registry,
    capabilities=[
        SystemPromptCapability("""You are an expert inventory manager.
        When shown items:
        1. Identify the item type and key features
        2. Assess condition from visual cues
        3. Estimate market value based on condition and features
        4. Maintain organized records with consistent descriptions

        Always be thorough in descriptions and conservative in value estimates.""")
    ]
)
```

---

## 💬 Using the System

Users can interact with the system in natural language:

```python
# Example interaction
result = agent.run("""I have a pair of Air Jordan basketball shoes.
                     They're red with the Jumpman logo, showing some wear
                     and slight discoloration.""")
```

### 🧠 Agent Might Respond:

> "I'll help you add those shoes to inventory.
>
> First, let me analyze the item details you've provided:
>
> * Item: Air Jordan Basketball Shoes
> * Color: Red
> * Notable Features: Jumpman logo
> * Condition: Used with visible wear and discoloration
>
> Based on these details and current market values, I'll create an inventory entry.
>
> **Action: save\_item**
>
> ```json
> {
>   "item_name": "Air Jordan Basketball Shoes",
>   "description": "Red colorway with iconic Jumpman logo",
>   "condition": "Used - visible wear and slight discoloration",
>   "estimated_value": 85.00
> }
> ```
>
> The shoes have been added to inventory. Would you like to add any additional items?"

---

## 🖼️ Extending with Images (Future Enhancement)

While not covered in our current implementation, this system could be extended to handle images with additional tools:

```python
@register_tool(description="Analyze an image and describe what you see")
def process_inventory_image(action_context: ActionContext,
                            image_path: str) -> str:
    """
    Look at an image and describe the item, including type, condition, and notable features.
    Returns a natural language description.
    """
    with open(image_path, "rb") as image_file:
        image_data = base64.b64encode(image_file.read()).decode("utf-8")

    response = completion(
        model="gpt-4o",
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": """Please describe this item for inventory purposes.
                        Include details about:
                        - What the item is
                        - Its key features
                        - The condition it's in
                        - Any visible wear or damage
                        - Anything notable about it"""
                    },
                    {
                        "type": "image",
                        "image_url": {
                            "url": f"data:image/jpeg;base64,{image_data}"
                        }
                    }
                ]
            }
        ],
        max_tokens=1000
    )

    return response
```

---

## ✅ Why This Approach Works

* **Simple Tools, Complex Understanding**:
  The tools handle basic operations while the LLM provides sophisticated analysis and decision-making.





## ✅ Why This Approach Works

This architecture succeeds because:

### 🧰 Simple Tools, Complex Understanding

The tools handle basic operations while the LLM provides sophisticated analysis and decision-making.

### 🗣️ Natural Interaction

Users can describe items in natural language or with photos instead of filling out structured forms.

### 🧠 Flexible Intelligence

The agent can:

* Identify items from descriptions
* Assess condition based on details provided
* Estimate values using market knowledge
* Maintain consistent record formats

### 🔧 Easy Extension

New capabilities can be added by:

* Updating the system prompt
* Adding simple tools
* Enhancing the agent’s goals

---

## 🌍 Real-World Applications

This pattern extends beyond inventory management. Consider:

* ✅ Policy compliance checking (like the travel expenses example)
* 📄 Document processing systems
* 🧑‍💼 Customer service applications
* 📊 Data analysis tools

---

## 🧱 The Key Principles

* **Define clear goals and instructions**
* **Provide simple, focused tools**
* **Let the agent’s intelligence handle complexity and allow adaptation**

---

> The future of software development isn’t about writing every possible edge case – it’s about providing the right framework for AI agents to handle complexity while keeping the tools and infrastructure simple and reliable.


